In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 2.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=896587f1d178a0bf6b8df7309f01bba8bb9247f991734c9089bb0b4fd1318d2f
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=8dd1f185ecf93cb103626800d797856f3c5f7034b7af50402321885864f682f8
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [9]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [11]:
train_images=train_images/255.0
test_images=test_images/255.0

In [13]:
train_images[0].shape

(28, 28)

In [15]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [17]:
def build_model(hp):
   model = keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=32,max_value=128, step=16),
          kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=64, step=16),
          kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
          activation='relu' 
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_filter',min_value=32,max_value=128, step=16),
          activation='relu' 
      ),
      keras.layers.Dense(10, activation='softmax')#output layer                    
   ])

   model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
               loss='sparse_categorical_crossentropy',  
               metrics=['accuracy'])
   return model

In [20]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [21]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [23]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 21m 33s]
val_accuracy: 0.8021666407585144

Best val_accuracy So Far: 0.8373333215713501
Total elapsed time: 01h 21m 36s
INFO:tensorflow:Oracle triggered exit


In [26]:
model=tuner_search.get_best_models(num_models=1)[0]

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        18464     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                589856    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 609,290
Trainable params: 609,290
Non-trainable params: 0
_________________________________________________________________
